# Important note!

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your GT login and the GT logins of any of your collaborators below. (The GT logins are worth 1 point per notebook, so don't miss the opportunity to get a free point!)

In [ ]:
YOUR_ID = "" # Please enter your GT login, e.g., "rvuduc3" or "gtg911x"
COLLABORATORS = [] # list of strings of your collaborators' IDs

In [ ]:
import re

RE_CHECK_ID = re.compile (r'''[a-zA-Z]+\d+|[gG][tT][gG]\d+[a-zA-Z]''')
assert RE_CHECK_ID.match (YOUR_ID) is not None

collab_check = [RE_CHECK_ID.match (i) is not None for i in COLLABORATORS]
assert all (collab_check)

del collab_check
del RE_CHECK_ID
del re

**Jupyter / IPython version check.** The following code cell verifies that you are using the correct version of Jupyter/IPython.

In [ ]:
import IPython
assert IPython.version_info[0] >= 3, "Your version of IPython is too old, please update it."

# Part B: Generating power law distributions

When studying dynamical systems on networks, we saw cases in which the structure of the graph played a role through the structure of the adjacency matrix as well as the _degree_ of every node. Thus, we might want to know how certain structures and _degree distributions_ arise.

This notebook considers one such generative process, known as the _preferential attachment_ model. The model has been rediscovered several times; the variant considered here is due to [Barabási and Albert (1999)](http://science.sciencemag.org/content/286/5439/509).

## Setup: A real-world network

Let's start by downloading an engineered network system, namely, the graph corresponding to the topology of internet routers: https://snap.stanford.edu/data/as-skitter.html

In [ ]:
import random
import numpy as np
import scipy.sparse as sps

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Download data
!mkdir -p nets
!wget https://github.com/rvuduc/cx4230sp17labs/blob/master/lab10/as-skitter.zip?raw=true -O nets/as-skitter.zip
    
# Unpack
!unzip nets/as-skitter.zip -d nets

In [ ]:
def read_net (name, fn_net=None, verbose=True, force_undir=True):
    if not fn_net:
        fn_net = 'nets/%s.txt' % name
        
    # Extract network file from archive as file-like object
    f_net = open (fn_net, 'rt') ; assert f_net
    
    # Read header
    line_num = 0
    while f_net:
        line_bytes = f_net.readline ()
        if not line_bytes:
            break
        line_num += 1
        
        line_text = line_bytes.strip ()
        if line_text[0] == '#': # header line
            if verbose:
                print ('%d: %s' % (line_num, line_text))
        else:
            break
            
    # Network data follows as a list of vertex pairs.
    I, J = [], []
            
    # EOF or first read line
    while line_text:
        fields = line_text.split (sep='\t')
        if len (fields) != 2:
            print ("*** Error reading line %d: '%s'" % (line_num, line_text))
            assert len (fields) == 2
            
        source, target = int (fields[0]), int (fields[1])
        I.append (source) ; J.append (target)
        if force_undir:
            J.append (source) ; I.append (target)
        
        if not f_net: # EOF
            break
            
        line_bytes = f_net.readline ()
        if not line_bytes: # EOF
            break
            
        line_num += 1
        line_text = line_bytes.strip ()
            
    # File cleanup
    del f_net
        
    return I, J

I, J = read_net ('as-skitter')

In [ ]:
A_net = sps.coo_matrix (([1.0]*len (I), (I, J))).tocsr ()

In [ ]:
def spy (A, figsize=(6, 6), markersize=0.5):
    fig = plt.figure (figsize=figsize)
    plt.spy (A, markersize=markersize)
    plt.show ()
    
# Render graph as sparse matrix
spy (A_net)

In [ ]:
from collections import defaultdict
from itertools import accumulate

def degrees (A):
    return np.squeeze (np.asarray (A.sum (axis=0)))

def degree_dist (A):
    sparse_hist = defaultdict (int)
    for d in degrees (A):
        sparse_hist[d] += 1
    degs = sorted (sparse_hist.keys ())
    counts = [sparse_hist[d] for d in degs]
    return degs, counts

def plot_degree_dist (A, cumulative=False, fig=None, figsize=(7, 7)):
    degs, counts = degree_dist (A)
    if cumulative:
        total = sum (counts)
        orig_counts = counts
        for i, c in enumerate (accumulate (orig_counts)):
            counts[i] = total - c
    if not fig:
        fig = plt.figure (figsize=figsize)
        plt.axes().set_aspect('equal')
    plt.loglog (degs, counts, '*')
    plt.grid (True)
    plt.xlabel ("degree (log-scale)")
    plt.title ("count (log-scale)")

In [ ]:
# Make same spy plot, but with vertices reordered by decreasing degree
K_net = np.argsort (degrees (A_net))[::-1]
A_net_permuted = (A_net[K_net, :].tocsc ())[:, K_net]
spy (A_net_permuted, markersize=0.01)

In [ ]:
plot_degree_dist (A_net)

**Exercise 1** (2 points). What is the exponent of a power law distribution that might fit this data? That is, if $d$ is the degree, then for what $\alpha$ does the data best fit a curve of the form $\dfrac{1}{d^{\alpha}}$?

YOUR ANSWER HERE

## Preferential attachment model

The preferential attachment model uses the following process to generate an _undirected_ network of $n$ vertices.

Initially, the network has $n_0 < n$ vertices, numbered $\{0, 1, ..., n_0-1\}$ with no edges. Each of the remaining vertices is generated one at a time, starting at vertex $n_0$, according to the following process:

1. Let $i$ denote the new vertex; at the start, $i=n_0$.
2. Let the probability of choosing any vertex $j < i$ be proportional to $d_j$, where $d_j$ is the degree of vertex $j$.
> Initially, since the first $n_0$ vertices have no edges, let the initial probabilities be uniform, i.e., the probability of choosing $j$ is just $1 / n_0$.
3. Using this degree-weighted probability distribution, connect $i$ to exactly $c$ of the vertices less than $i$, chosen randomly without replacement.

**Exercise 2** (2 points). According to this process, how many edges will be produced?

YOUR ANSWER HERE

In [ ]:
N_INITIAL = 10 # Number of vertices, initially
N_FINAL = 10000 # Number of vertices, finally
C = 5 # Number of initial connections per new vertex

assert N_FINAL > N_INITIAL
assert C <= N_INITIAL

**Exercise 3** (4 points). Simulate the preferential attachment process and see if the degree distribution matches. In particular, write some code to produce a sparse matrix `A` (in CSR format) that holds the adjacency matrix of a graph generated by a preferential attachment process. That is, $a_{ij} = 1$ if there is a _directed_ edge $(i, j)$, or 0 otherwise; and since the final graph should be undirected, $a_{ij} = 1$ means $a_{ji} = 1$, too.

> Hint 1: You may find [`numpy.random.choice()`](http://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.random.choice.html) handy.
>
> Hint 2: Create a CSR matrix from row indices `row_ind[:]`, column indices `col_ind[:]`, and values `val[:]` using:
>
>     sps.coo_matrix ((vals, (row_ind, col_ind))).tocsr ()

In [ ]:
# Indices of all N_FINAL vertices
vertices = np.arange (N_FINAL)

# Create a network with M0 vertices
degree = np.zeros (N_FINAL) # degree[i] = degree of vertex i
prob = np.zeros (N_FINAL) # prob[i] = probability of linking to vertex i

# Initially, let each of the initial vertices be equally likely
prob[:N_INITIAL] = 1.0 / N_INITIAL

# Maintain a list of directed edges, {(sources[k], targets[k])},
# initially empty
M_FINAL = (N_FINAL - N_INITIAL) * C * 2 # max number of edges
sources = -np.ones (M_FINAL)
targets = -np.ones (M_FINAL)
edges = -np.ones ((M_FINAL, 2))

m = 0 # number of edges so far
for i in range (N_INITIAL, N_FINAL):
    # @YOUSE: Your solution here.
    # Suggested steps:
    # 1. Generate neighbors -- see np.random.choice()
    # 2. Record edges and update their number (m)
    # 3. Update probabilities

    # YOUR CODE HERE
    raise NotImplementedError()
    
val = np.ones (M_FINAL)
A = sps.coo_matrix ((val, (sources, targets))).tocsr ()

In [ ]:
spy (A, markersize=0.1)

In [ ]:
plot_degree_dist (A)

**Exercise 4** (2 points). Run the above for $n=10,000$. What is the exponent of the power law distribution that best fits this data?

YOUR ANSWER HERE